In [23]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math

In [2]:
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time

In [3]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [163]:
# Get season stats for 16-17 season to get starting elo for training data
nba_16_17_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2016-17&SeasonType=Regular%20Season"

In [164]:
# visit website
browser.visit(nba_16_17_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [165]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [166]:
df_init = df.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
game_dates = df_init['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]

In [167]:
date_index=0
newElo = {}
eloScores = []
for row in df_init.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
    newElo[row.Team] = eloUpdate
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [169]:
db.nba_team_elo_training.insert_one(elo_list)

In [4]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_trad_boxscores_url = "https://stats.nba.com/teams/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2017-18&SeasonType=Regular%20Season"

In [8]:
# create splinter browser
def init_browser(noShowBrowser):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=noShowBrowser)
    return browser

In [224]:
browser = init_browser(False)

In [225]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df_adv = df[0].drop(columns=['Season'])
# Rename column names
df_adv.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df_adv['GameDate'] = df_adv['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [226]:
df_adv.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE'],
      dtype='object')

In [229]:
# visit website
browser.visit(nba_17_18_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_ff = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%', 'OppeFG%', 'OppFTARate', 'OppTOV%', 'OppOREB%'], axis=1)

In [230]:
df_ff.columns

Index(['Team', 'GameDate', 'FTARate'], dtype='object')

In [232]:
# visit website
browser.visit(nba_17_18_trad_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_trad = df.drop(['MIN', 'MatchUp', 'W/L', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', '+/-'], axis=1)

In [233]:
df_trad.columns

Index(['Team', 'GameDate', 'PTS'], dtype='object')

In [234]:
result = pd.merge(df_adv, df_ff, on=['Team', 'GameDate'])
result.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate'],
      dtype='object')

In [235]:
result_final = pd.merge(result, df_trad, on=['Team', 'GameDate'])
result_final.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'PTS'],
      dtype='object')

In [28]:
teams = result_final['Team'].unique()

In [29]:
myteam = [team for team in teams]

In [30]:
myteam.sort()

In [236]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [37]:
db.nba_team_elo_training.insert_one(elo_list)

In [261]:
result_sorted = result_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
#result_sorted['HomeGame'] = result_sorted['MatchUp'].apply(lambda x: 1 if x.split(' ')[1] == 'vs.' else 0)

In [64]:
def eloProbability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
def EloRating(homeTeam, roadTeam, K, homeTeamWins):
    # To calculate the Winning 
    # Probability of Player B 
    Pb = eloProbability(homeTeam, roadTeam) 

    # To calculate the Winning 
    # Probability of Player A 
    Pa = eloProbability(roadTeam, homeTeam) 

    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (homeTeamWins == True) : 
        homeTeam = homeTeam + K * (1 - Pa) 
        roadTeam = roadTeam + K * (0 - Pb) 


    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    else : 
        homeTeam = homeTeam + K * (0 - Pa)
        roadTeam = roadTeam + K * (1 - Pb)
    return homeTeam, roadTeam

In [66]:
team1 = 1000
team2 = 1000

In [67]:
team1, team2 = EloRating(team1, team2, 20, False)

In [69]:
team2

1010.0

In [262]:
game_dates = result_sorted['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]
game_dates[0]

numpy.datetime64('2017-10-17T00:00:00.000000000')

In [81]:
game_dates[1] == np.datetime64(result_sorted.iloc[0]['GameDate'])

False

In [76]:
result_sorted.iloc[0]['GameDate']

Timestamp('2017-10-17 00:00:00')

In [264]:
temp_df = result_sorted.copy()

In [265]:
temp_df.dtypes

Team                object
MatchUp             object
GameDate    datetime64[ns]
W/L                 object
MIN                  int64
OffRtg             float64
DefRtg             float64
NetRtg             float64
AST%               float64
AST/TO             float64
ASTRatio           float64
OREB%              float64
DREB%              float64
REB%               float64
TOV%               float64
eFG%               float64
TS%                float64
PACE               float64
PIE                float64
FTARate            float64
PTS                  int64
dtype: object

In [266]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [267]:
date_index=0
newElo = {}
eloScores = []
homeTeam = []
roadTeam = []
for row in temp_df.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
        homeTeam.append(team1)
        roadTeam.append(team2)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
        roadTeam.append(team1)
        homeTeam.append(team2)
        
    newElo[row.Team] = eloUpdate
    eloScores.append(eloUpdate)
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [268]:
elo_list

{'ATL': 1379.585954026032,
 'BKN': 1404.8281020435775,
 'BOS': 1581.6442164590449,
 'CHA': 1455.7326971389414,
 'CHI': 1369.5264989320076,
 'CLE': 1569.2570084824733,
 'DAL': 1349.9955539253224,
 'DEN': 1567.3908510170115,
 'DET': 1469.2436740169394,
 'GSW': 1585.218097082357,
 'HOU': 1712.3692616813594,
 'IND': 1579.9535417247766,
 'LAC': 1514.4717110441677,
 'LAL': 1463.3333766030464,
 'MEM': 1315.91711181331,
 'MIA': 1515.8396295778298,
 'MIL': 1518.023873793636,
 'MIN': 1538.5720049287456,
 'NOP': 1588.4605069805805,
 'NYK': 1369.6778447449904,
 'OKC': 1579.0910067247698,
 'ORL': 1352.7914611902586,
 'PHI': 1638.0820435490868,
 'PHX': 1305.2825248358438,
 'POR': 1592.5772517125845,
 'SAC': 1399.0131403590187,
 'SAS': 1541.9868737873699,
 'TOR': 1642.7968580395868,
 'UTA': 1613.6622117595898,
 'WAS': 1485.675112025742}

In [245]:
len(eloScores)

2460

In [246]:
len(temp_df)

2460

In [269]:
temp_df['ELO'] = eloScores
temp_df['HomeTeam'] = homeTeam
temp_df['RoadTeam'] = roadTeam

In [270]:
temp_df.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'PTS', 'ELO',
       'HomeTeam', 'RoadTeam'],
      dtype='object')

In [271]:
home_df = temp_df[temp_df['Team'] == temp_df['HomeTeam']]
road_df = temp_df[temp_df['Team'] == temp_df['RoadTeam']]

In [278]:
game_df = pd.merge(home_df, road_df, how='inner', on=['HomeTeam', 'RoadTeam', 'GameDate'], suffixes=('_h', '_r'), validate="one_to_one")

In [279]:
game_df.columns

Index(['Team_h', 'MatchUp_h', 'GameDate', 'W/L_h', 'MIN_h', 'OffRtg_h',
       'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h', 'ASTRatio_h', 'OREB%_h',
       'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h', 'TS%_h', 'PACE_h', 'PIE_h',
       'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam', 'RoadTeam', 'Team_r',
       'MatchUp_r', 'W/L_r', 'MIN_r', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r',
       'AST%_r', 'AST/TO_r', 'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r',
       'TOV%_r', 'eFG%_r', 'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r',
       'ELO_r'],
      dtype='object')

In [280]:
game_df = game_df.drop(['Team_h', 'Team_r', 'MatchUp_h', 'MatchUp_r', 'W/L_h', 'W/L_r', 'MIN_h', 'MIN_r'], axis=1)

In [282]:
game_df.columns

Index(['GameDate', 'OffRtg_h', 'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h',
       'ASTRatio_h', 'OREB%_h', 'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h',
       'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam',
       'RoadTeam', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [283]:
db.score_pred_training_data.drop()

In [284]:
# export to json
items = game_df.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [285]:
db.score_pred_training_data.insert_many(items_db)

In [185]:
db.nba_team_elo_testing.insert_one(elo_list)

In [186]:
temp = db.nba_team_elo_testing.find()

In [188]:
temp = list(temp)

In [192]:
for i in temp:
    i.pop('_id', None)

In [194]:
elo_testing = temp[0]

In [220]:
elo_testing

{'ATL': 1375.7075741834688,
 'BKN': 1385.7473632514852,
 'BOS': 1593.1622149515758,
 'CHA': 1447.7280767309485,
 'CHI': 1367.1005274309066,
 'CLE': 1570.4270859757785,
 'DAL': 1343.0240681009527,
 'DEN': 1571.0319534263213,
 'DET': 1461.7873401532459,
 'GSW': 1619.4570676450153,
 'HOU': 1727.4508941343163,
 'IND': 1581.231834171123,
 'LAC': 1524.2948631384638,
 'LAL': 1451.6639776614604,
 'MEM': 1312.794183162584,
 'MIA': 1518.578999882534,
 'MIL': 1518.6617359287,
 'MIN': 1533.1207602745035,
 'NOP': 1588.8232069597443,
 'NYK': 1356.0233382357949,
 'OKC': 1586.408713208459,
 'ORL': 1336.8270182401138,
 'PHI': 1624.0357372854346,
 'PHX': 1286.05099854899,
 'POR': 1598.9654622637888,
 'SAC': 1390.6202815200002,
 'SAS': 1558.4988631578274,
 'TOR': 1652.5998449378783,
 'UTA': 1626.4434841314753,
 'WAS': 1491.7325313071099}

In [196]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [286]:
game_df.columns

Index(['GameDate', 'OffRtg_h', 'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h',
       'ASTRatio_h', 'OREB%_h', 'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h',
       'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam',
       'RoadTeam', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [438]:
#X_train = game_df.loc[:,['OffRtg_h', 'DefRtg_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
#                         'OffRtg_r', 'DefRtg_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
X_train = game_df.loc[:,['PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
                         'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
y_h = game_df['PTS_h']
y_r = game_df['PTS_r']

In [439]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

In [299]:
homeScorePredictor = LinearRegression()
roadScorePredictor = LinearRegression()

In [300]:
homeScorePredictor.fit(X_train_scaled, y_h)
roadScorePredictor.fit(X_train_scaled, y_r)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [301]:
homeScorePredictor.score(X_train_scaled, y_h)

0.8589936215712799

In [302]:
roadScorePredictor.score(X_train_scaled, y_r)

0.8675338273816895

In [303]:
homeScorePredictor.coef_

array([ 2.58503402e+02, -1.62216669e-01,  2.76297419e+01, -4.00828910e+00,
        8.07205963e+00,  2.05972202e+00,  2.10898674e-01,  2.57524054e+02,
        3.92920522e-01,  2.52358929e+01,  1.75240593e-01,  1.45243507e+00,
        2.05972202e+00, -1.01268017e-01])

In [304]:
roadScorePredictor.coef_

array([ 2.37036741e+02,  2.52750004e-01, -1.79132152e+01,  2.25720489e-01,
        1.26837505e+00,  2.04936389e+00, -8.64979886e-02,  2.37786874e+02,
       -3.02186776e-02, -1.53423093e+01, -4.02832807e+00,  8.62107714e+00,
        2.04936389e+00,  2.39525620e-01])

In [444]:
from sklearn.linear_model import BayesianRidge

In [448]:
homeBay = BayesianRidge(verbose=1)
roadBay = BayesianRidge(verbose=1)

In [449]:
X_train_scaled.shape

(802, 14)

In [450]:
homeBay.fit(X_train_scaled, y_h)
roadBay.fit(X_train_scaled, y_r)

Convergence after  4  iterations
Convergence after  4  iterations


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=1)

In [463]:
homeBay.score(X_train_scaled, y_h)

0.8524515534343657

In [464]:
roadBay.score(X_train_scaled, y_r)

0.8436031702629764

# Create Testing Data

In [305]:
# Url to get all boxscores for 2017-2018 regular season games
nba_18_19_trad_boxscores_url = "https://stats.nba.com/teams/boxscores-traditional/"
nba_18_19_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/"
nba_18_19_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/"

In [306]:
# visit website
browser.visit(nba_18_19_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df_adv = df[0].drop(columns=['Season'])
# Rename column names
df_adv.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df_adv['GameDate'] = df_adv['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_adv = df_adv.drop(['MIN'], axis=1)

In [307]:
# visit website
browser.visit(nba_18_19_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_ff = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%', 'OppeFG%', 'OppFTARate', 'OppTOV%', 'OppOREB%'], axis=1)

In [308]:
# visit website
browser.visit(nba_18_19_trad_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_trad = df.drop(['MIN', 'MatchUp', 'W/L', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', '+/-'], axis=1)

In [309]:
result_test = pd.merge(df_adv, df_ff, on=['Team', 'GameDate'])
result_test_final = pd.merge(result_test, df_trad, on=['Team', 'GameDate'])

In [310]:
result_test_sorted = result_test_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)

In [311]:
game_dates = result_test_sorted['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]

In [313]:
temp_df = result_test_sorted.copy()

In [314]:
temp = db.nba_team_elo_testing.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
elo_testing = temp[0]

In [315]:
date_index=0
newElo = {}
eloScores = []
homeTeam = []
roadTeam = []
for row in temp_df.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_testing[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_testing[team1], elo_testing[team2], 20, outcome)
        homeTeam.append(team1)
        roadTeam.append(team2)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_testing[team2], elo_testing[team1], 20, outcome)
        roadTeam.append(team1)
        homeTeam.append(team2)
        
    newElo[row.Team] = eloUpdate
    eloScores.append(eloUpdate)
# Update for last date
for key in newElo:
    elo_testing[key] = newElo[key]
# Should update ELO Testing DB

In [316]:
temp_df['ELO'] = eloScores
temp_df['HomeTeam'] = homeTeam
temp_df['RoadTeam'] = roadTeam

In [318]:
home_df = temp_df[temp_df['Team'] == temp_df['HomeTeam']]
road_df = temp_df[temp_df['Team'] == temp_df['RoadTeam']]

In [319]:
game_df = pd.merge(home_df, road_df, how='inner', on=['HomeTeam', 'RoadTeam', 'GameDate'], suffixes=('_h', '_r'), validate="one_to_one")

In [320]:
game_df.columns

Index(['Team_h', 'MatchUp_h', 'GameDate', 'W/L_h', 'OffRtg_h', 'DefRtg_h',
       'NetRtg_h', 'AST%_h', 'AST/TO_h', 'ASTRatio_h', 'OREB%_h', 'DREB%_h',
       'REB%_h', 'TOV%_h', 'eFG%_h', 'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h',
       'PTS_h', 'ELO_h', 'HomeTeam', 'RoadTeam', 'Team_r', 'MatchUp_r',
       'W/L_r', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [321]:
game_df = game_df.drop(['Team_h', 'Team_r', 'MatchUp_h', 'MatchUp_r', 'W/L_h', 'W/L_r'], axis=1)

In [334]:
game_df.tail()

,GameDate,OffRtg_h,DefRtg_h,NetRtg_h,AST%_h,AST/TO_h,ASTRatio_h,OREB%_h,DREB%_h,REB%_h,...,DREB%_r,REB%_r,TOV%_r,eFG%_r,TS%_r,PACE_r,PIE_r,FTARate_r,PTS_r,ELO_r
797,2019-02-05,134.7,92.2,42.5,66.1,3.70,24.7,26.7,82.9,53.5,...,73.3,46.5,18.6,49.3,54.4,101.5,34.8,0.347,94,1467.988589
798,2019-02-05,117.3,121.9,-4.5,48.7,3.80,14.6,26.8,78.3,50.0,...,73.2,50.0,11.5,58.6,61.9,97.0,51.2,0.195,117,1517.404605
799,2019-02-05,103.2,109.6,-6.3,52.5,1.50,15.6,36.8,78.6,54.5,...,63.2,45.5,13.8,54.3,58.4,93.5,60.9,0.171,103,1615.457081
800,2019-02-05,116.8,108.0,8.8,54.3,1.47,16.6,28.0,83.0,56.3,...,72.0,43.7,15.9,52.8,58.1,113.0,46.0,0.378,122,1403.429442
801,2019-02-05,113.7,110.4,3.3,50.0,2.25,14.3,24.1,59.2,40.8,...,75.9,59.2,17.7,50.0,55.3,95.5,51.9,0.230,106,1488.293300


In [323]:
# export to json
items = game_df.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [324]:
db.score_pred_testing_data.insert_many(items_db)

In [325]:
X_test = game_df.loc[:,['PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
                         'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
y_h = game_df['PTS_h']
y_r = game_df['PTS_r']

In [370]:
X_test.shape

(802, 14)

In [326]:
X_test_scaled = X_scaler.transform(X_test)

In [327]:
homeScorePredictor.score(X_test_scaled, y_h)

0.8319517712939757

In [328]:
roadScorePredictor.score(X_test_scaled, y_r)

0.830255034316016

In [332]:
predictions_h = homeScorePredictor.predict(X_test_scaled)

In [330]:
predictions[0]

102.24064642117432

In [335]:
y_h[797]

136

In [336]:
predictions_r = roadScorePredictor.predict(X_test_scaled)

In [337]:
predictions_h[797]

132.77524310276573

In [338]:
predictions_r[797]

100.9754252279464

In [339]:
y_r[797]

94

In [344]:
#lakers_df = game_df[np.logical_or(game_df['HomeTeam'] == 'LAL', game_df['RoadTeam'] == 'LAL')]

In [348]:
lakers_df = game_df[game_df['RoadTeam'] == 'LAL']

In [349]:
indiana_df = game_df[game_df['HomeTeam'] == 'IND']

In [350]:
#result_test_sorted = result_test_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
lakers_df = lakers_df.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
indiana_df = indiana_df.sort_values(['GameDate'], ascending=0).reset_index(drop=True)

In [453]:
laker_mean = lakers_df[1:6].mean()

In [433]:
laker_mean.iloc[2]

GameDate      2019-01-19 00:00:00
OffRtg_h                    124.3
DefRtg_h                    119.6
NetRtg_h                      4.7
AST%_h                       54.8
AST/TO_h                     1.92
ASTRatio_h                   15.3
OREB%_h                      27.4
DREB%_h                      65.3
REB%_h                       44.1
TOV%_h                       10.8
eFG%_h                       54.5
TS%_h                        59.6
PACE_h                     100.98
PIE_h                          45
FTARate_h                    0.36
PTS_h                         138
ELO_h                     1630.34
HomeTeam                      HOU
RoadTeam                      LAL
OffRtg_r                    119.6
DefRtg_r                    124.3
NetRtg_r                     -4.7
AST%_r                         54
AST/TO_r                     1.69
ASTRatio_r                   17.4
OREB%_r                      34.7
DREB%_r                      72.6
REB%_r                       55.9
TOV%_r        

In [454]:
indiana_mean = indiana_df[1:6].mean()

In [401]:
laker_mean['OffRtg_h']

134.7

In [455]:
my_test = pd.Series(data = [indiana_mean['PIE_h'], indiana_mean['FTARate_h'], indiana_mean['REB%_h'], indiana_mean['TOV%_h'], indiana_mean['TS%_h'], indiana_mean['PACE_h'], indiana_mean['ELO_h'],
                            laker_mean['PIE_r'], laker_mean['FTARate_r'], laker_mean['REB%_r'], laker_mean['TOV%_r'], laker_mean['TS%_r'], laker_mean['PACE_r'], laker_mean['ELO_r']
                           ])
my_test

0       48.700000
1        0.180800
2       48.380000
3       10.920000
4       53.240000
5       98.500000
6     1603.494912
7       48.980000
8        0.263000
9       53.460000
10      15.240000
11      54.600000
12     103.394000
13    1483.664565
dtype: float64

In [417]:
my_test = pd.Series(data = [indiana_mean['PIE_h'], indiana_mean['FTARate_h'], indiana_mean['REB%_h'], indiana_mean['TOV%_h'], indiana_mean['TS%_h'], indiana_mean['PACE_h'], indiana_mean['ELO_h'],
                            laker_mean['PIE_r'], laker_mean['FTARate_r'], laker_mean['REB%_r'], laker_mean['TOV%_r'], laker_mean['TS%_r'], laker_mean['PACE_r'], laker_mean['ELO_r']
                           ])
my_test

0       48.700000
1        0.180800
2       48.380000
3       10.920000
4       53.240000
5       98.500000
6     1603.494912
7       48.980000
8        0.263000
9       53.460000
10      15.240000
11      54.600000
12     103.394000
13    1483.664565
dtype: float64

In [456]:
my_test = my_test.values.reshape(1,14)

In [457]:
X_my_scaled = X_scaler.transform(my_test)

In [469]:
homeScore = homeScorePredictor.predict(X_my_scaled)

In [470]:
homeScore[0]

223.93688820520185

In [471]:
roadScorePredictor.predict(X_my_scaled)[0]

200.0637713494899

In [393]:
lakers_df[0:1]['PIE_h']

0    65.2
Name: PIE_h, dtype: float64

In [539]:
homeBay.predict(X_my_scaled)[0]

107.59452499218354

In [473]:
roadBay.predict(X_my_scaled)

array([111.39097325])

In [465]:
def getHomeRoadMeans(teamHome, teamRoad):
    temp = db.score_pred_testing_data.find({'$or' : [{'HomeTeam': teamHome}, {'RoadTeam': teamRoad}]})
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df_test = pd.DataFrame(temp)
    df_test = df_test.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
    home_df = df_test[df_test['HomeTeam'] == teamHome].reset_index(drop=True)
    road_df = df_test[df_test['RoadTeam'] == teamRoad].reset_index(drop=True)
    homeMean = home_df[0:5].mean()
    roadMean = road_df[0:5].mean()
    test_series = pd.Series(data = [homeMean['PIE_h'], homeMean['FTARate_h'], homeMean['REB%_h'], homeMean['TOV%_h'], homeMean['TS%_h'], homeMean['PACE_h'], homeMean['ELO_h'],
                                    roadMean['PIE_r'], roadMean['FTARate_r'], roadMean['REB%_r'], roadMean['TOV%_r'], roadMean['TS%_r'], roadMean['PACE_r'], roadMean['ELO_r']
                           ])
    test_data = test_series.values.reshape(1,14)
    return test_data

In [497]:
X_test = getHomeRoadMeans('UTA', 'HOU')

In [498]:
X_test

array([[4.83200000e+01, 3.40000000e-01, 4.99200000e+01, 1.42400000e+01,
        5.48600000e+01, 9.86000000e+01, 1.59891654e+03, 5.11400000e+01,
        2.64000000e-01, 5.01800000e+01, 1.51600000e+01, 5.76000000e+01,
        9.87000000e+01, 1.68651798e+03]])

In [499]:
X_my_scaled = X_scaler.transform(X_test)

In [500]:
homeBay.predict(X_my_scaled)[0]

107.59452499218354

In [501]:
roadBay.predict(X_my_scaled)[0]

110.73290017421905

In [503]:
import pickle
from sklearn.externals import joblib

In [504]:
scaler_filename = "bay_rig_scaler.save"
joblib.dump(X_scaler, scaler_filename)
bay_rig_home_filename = 'bay_rig_home_model.pkl'
pickle.dump(homeBay, open(bay_rig_home_filename, 'wb'))
bay_rig_road_filename = 'bay_rig_road_model.pkl'
pickle.dump(roadBay, open(bay_rig_road_filename, 'wb'))


In [537]:
teamHome = 'LAL'
teamRoad = 'LAC'
temp = db.score_pred_testing_data.find({'$or' : [{'HomeTeam': teamHome}, {'RoadTeam': teamRoad}]})
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_test = pd.DataFrame(temp)
df_test = df_test.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
home_df = df_test[df_test['HomeTeam'] == teamHome].reset_index(drop=True)
road_df = df_test[df_test['RoadTeam'] == teamRoad].reset_index(drop=True)

In [532]:
temp = home_df.head()['ELO_h']

In [538]:
home_df.iloc[0]['ELO_h']

1468.8400819865

In [529]:
temp

level_0       0.000000
index         3.000000
ELO_h      1468.840082
Name: 0, dtype: float64

In [540]:
bayScalar_filename = "bay_rig_scaler.save"
bay_rig_scaler = joblib.load(bayScalar_filename)

In [ ]:
bay_rig_scaler.scale_